**TRAINING CLASSIFICATION MODEL**

---



**Platform**: Colab

**Requirement**: GPU

**Written on**: 14 September 2021

**Tested on**: 22 September 2021

**Author**: A.S. Faraz Ahmed

**Description**: 

> Train Classifier

# Setup

In [ ]:
import os
import shutil
os.chdir('/content')
if "sample_data" in os.listdir():
  shutil.rmtree('sample_data/')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Classification

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
import matplotlib.image as mpimg
import numpy as np

In [ ]:
class Classifier:
  def __init__(self, data_dir, class_names, save_model):
    self.data_dir = data_dir
    self.class_names = class_names
    self.save_model = save_model

    self.image_size = (75, 75)
    self.batch_size = 128
    self.dataset_dir = "/content/training/"

  def preprocesss(self):
    self.train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      pathlib.Path(self.dataset_dir),
      validation_split=0.2,
      subset="training",
      seed=1337,
      image_size=self.image_size,
      batch_size=self.batch_size,
      shuffle=True,
    )
    self.val_ds = tf.keras.preprocessing.image_dataset_from_directory(
      pathlib.Path(self.dataset_dir),
      validation_split=0.2,
      subset="validation",
      seed=1337,
      image_size=self.image_size,
      batch_size=self.batch_size,
      shuffle=True,
    )
    self.class_names = self.train_ds.class_names
    print("Class Names: ",self.class_names)

    self.num_classes = len(self.class_names)
    print("Number of Classes: ",self.num_classes)

  def visualize(self):
    plt.figure(figsize=(10, 10))
    for images, labels in self.train_ds.take(1):
      for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(self.class_names[labels[i]])
        plt.axis("off")

  def train(self):
    IMG_RESIZE = 75
    resize_and_rescale = tf.keras.Sequential([
      layers.experimental.preprocessing.Resizing(IMG_RESIZE, IMG_RESIZE),
      layers.experimental.preprocessing.Rescaling(1./255)
    ])
    self.model = Sequential([
      resize_and_rescale,
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(self.num_classes,activation='softmax')
    ])
    self.model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
    epochs=10
    history = self.model.fit(
      self.train_ds,
      validation_data=self.val_ds,
      epochs=epochs
    )
    self.model.summary()
    self.model.save(self.save_model)

  def evaluate(self, image, labels):
    # image = mpimg.imread(image_path)
    image = keras.preprocessing.image.load_img(
        image, target_size=(self.image_size[0], self.image_size[1])
    )
    test_loss, test_acc = self.model.evaluate(image,  labels, verbose=2)
    print("Test Loss: {}, Test Accuracy: {}".format(test_loss, test_acc))

  def test(self, image_path):
    self.test_model = tf.keras.models.load_model(self.save_model)
    img_height = 100
    img_width = 100

    img = mpimg.imread(image_path)
    imgplot = plt.imshow(img)
    plt.show()
        
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(self.image_size[0], self.image_size[1])
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = self.test_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(self.class_names[np.argmax(score)], 100 * np.max(score))
    )

  def move_files(self):
    os.system("mkdir /content/training/")
    for folder in self.class_names:
      src = self.data_dir + "/" + folder
      if os.path.exists(src):
        des = " /content/training/"
        os.system("cp -r "+src+des)

  def remove_files(self):
    os.system("rm -r training/")



# Raw Image

In [ ]:
%%bash
cp "/content/gdrive/MyDrive/dataset/Real Time Motion Control of a Humanoid Robot Using Deep Learnin/raw_with_test.tar.gz" ./
tar -xvzf raw_with_test.tar.gz

In [ ]:
for folder in os.listdir("raw"):
  print(folder, ":", len(os.listdir("raw/"+folder)))

rightup : 2400
left : 2400
up : 2400
leftup : 2400
right : 2400
down : 2400
full : 2400
test : 690


In [ ]:
%%bash
mv raw/test ./
rm raw_with_test.tar.gz

## 2 Class

In [ ]:
two_classes = Classifier("/content/raw", ["up", "down"], "raw_2.h5")
two_classes.move_files()
two_classes.preprocesss()
two_classes.visualize()
two_classes.train()
two_classes.evaluate("/content/test/100.jpg", "up")
two_classes.test("/content/test/100.jpg") #up
two_classes.test("/content/test/1.jpg") #down
two_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_2class_with_test.tar.gz ./raw_2.h5 ./test/
!rm -r /content/raw_2.h5

In [ ]:
# UNZIP FILES
!tar -xvzf raw_2class_with_test.tar.gz ./

## 3 Class

In [ ]:
three_classes = Classifier("/content/raw", ["up", "down", "full"], "raw_3.h5")
three_classes.move_files()
three_classes.preprocesss()
three_classes.visualize()
three_classes.train()
three_classes.evaluate("/content/test/346.jpg", "full")
three_classes.test("/content/test/100.jpg") #up
three_classes.test("/content/test/5.jpg") #down
three_classes.test("/content/test/346.jpg") #full
three_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_3class_with_test.tar.gz ./raw_3.h5 ./test/
!rm -r /content/raw_3.h5

In [ ]:
# UNZIP FILES
!tar -xvzf raw_3class_with_test.tar.gz ./

## 4 Class

In [ ]:
four_classes = Classifier("/content/raw", ["up", "down", "full", "right"], "raw_4.h5")
four_classes.move_files()
four_classes.preprocesss()
four_classes.visualize()
four_classes.train()
four_classes.evaluate("/content/test/346.jpg", "full")
four_classes.test("/content/test/381.jpg") #up
four_classes.test("/content/test/518.jpg") #down
four_classes.test("/content/test/346.jpg") #full
four_classes.test("/content/test/463.jpg") #right
four_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_4class_with_test.tar.gz ./raw_4.h5 ./test/
!rm -r /content/raw_4.h5

In [ ]:
# UNZIP FILES
!tar -xvzf raw_4class_with_test.tar.gz ./

## 5 Class

In [ ]:
five_classes = Classifier("/content/raw", ["up", "down", "full", "right", "left"], "raw_5.h5")
five_classes.move_files()
five_classes.preprocesss()
five_classes.visualize()
five_classes.train()
five_classes.evaluate("/content/test/346.jpg", "full")
five_classes.test("/content/test/384.jpg") #up
five_classes.test("/content/test/5.jpg") #down
five_classes.test("/content/test/343.jpg") #full
five_classes.test("/content/test/501.jpg") #right
five_classes.test("/content/test/265.jpg") #left
five_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_5class_with_test.tar.gz ./skeleton_5.h5 ./test/
!rm -r /content/raw_5.h5

In [ ]:
# UNZIP FILES
!tar -xvzf raw_5class_with_test.tar.gz ./

## 6 Class

In [ ]:
six_classes = Classifier("/content/raw", ["up", "down", "full", "right", "left", "rightup"], "raw_6.h5")
six_classes.move_files()
six_classes.preprocesss()
six_classes.visualize()
six_classes.train()
six_classes.evaluate("/content/test/346.jpg", "full")
six_classes.test("/content/test/384.jpg") #up
six_classes.test("/content/test/5.jpg") #down
six_classes.test("/content/test/343.jpg") #full
six_classes.test("/content/test/501.jpg") #right
six_classes.test("/content/test/265.jpg") #left
six_classes.test("/content/test/405.jpg") #rightup
six_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_6class_with_test.tar.gz ./raw_6.h5 ./test/
!rm -r /content/raw_6.h5

# UNZIP FILES
!tar -xvzf raw_6class_with_test.tar.gz ./

## 7 Class

In [ ]:
seven_classes = Classifier("/content/raw", ["up", "down", "full", "right", "left", "rightup", "leftup"], "raw_7.h5")
seven_classes.move_files()
seven_classes.preprocesss()
seven_classes.visualize()
seven_classes.train()
seven_classes.evaluate("/content/test/346.jpg", "full")
seven_classes.test("/content/test/384.jpg") #up
seven_classes.test("/content/test/5.jpg") #down
seven_classes.test("/content/test/343.jpg") #full
seven_classes.test("/content/test/501.jpg") #right
seven_classes.test("/content/test/265.jpg") #left
seven_classes.test("/content/test/405.jpg") #rightup
seven_classes.test("/content/test/597.jpg") #leftup
seven_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf raw_7class_with_test.tar.gz ./raw_7.h5 ./test/
!rm -r /content/raw_7.h5

# UNZIP FILES
!tar -xvzf raw_7class_with_test.tar.gz ./

In [ ]:
265 555 561 630 641 left
381 384 100 up
394 405 rightup
463 501 right
577 582 597 leftup
5 518 662 down
346 343 full

# Pose Estimated

In [ ]:
%%bash
cp "/content/gdrive/MyDrive/dataset/Real Time Motion Control of a Humanoid Robot Using Deep Learnin/pose_estimate_with_test.tar.gz" ./
tar -xvzf pose_estimate_with_test.tar.gz

In [ ]:
for folder in os.listdir("raw"):
  print(folder, ":", len(os.listdir("pose_estimate/"+folder)))

In [ ]:
%%bash
mv pose_estimate/test ./
rm pose_estimate_with_test.tar.gz

## 2 Class

In [ ]:
two_classes = Classifier("/content/pose_estimate", ["up", "down"], "skeleton_2.h5")
two_classes.move_files()
two_classes.preprocesss()
two_classes.visualize()
two_classes.train()
two_classes.evaluate("/content/test/100.jpg", "up")
two_classes.test("/content/test/100.jpg") #up
two_classes.test("/content/test/1.jpg") #down
two_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_2class_with_test.tar.gz ./skeleton_2.h5 ./test/
!rm -r /content/skeleton_2.h5

In [ ]:
# UNZIP FILES
!tar -xvzf pose_estimate_2class_with_test.tar.gz ./

## 3 Class

In [ ]:
three_classes = Classifier("/content/pose_estimate", ["up", "down", "full"], "skeleton_3.h5")
three_classes.move_files()
three_classes.preprocesss()
three_classes.visualize()
three_classes.train()
three_classes.evaluate("/content/test/346.jpg", "full")
three_classes.test("/content/test/100.jpg") #up
three_classes.test("/content/test/5.jpg") #down
three_classes.test("/content/test/346.jpg") #full
three_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_3class_with_test.tar.gz ./skeleton_3.h5 ./test/
!rm -r /content/skeleton_3.h5

In [ ]:
# UNZIP FILES
!tar -xvzf pose_estimate_3class_with_test.tar.gz ./

## 4 Class

In [ ]:
four_classes = Classifier("/content/pose_estimate", ["up", "down", "full", "right"], "skeleton_4.h5")
four_classes.move_files()
four_classes.preprocesss()
four_classes.visualize()
four_classes.train()
four_classes.evaluate("/content/test/346.jpg", "full")
four_classes.test("/content/test/381.jpg") #up
four_classes.test("/content/test/518.jpg") #down
four_classes.test("/content/test/346.jpg") #full
four_classes.test("/content/test/463.jpg") #right
four_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_4class_with_test.tar.gz ./skeleton_4.h5 ./test/
!rm -r /content/skeleton_4.h5

In [ ]:
# UNZIP FILES
!tar -xvzf pose_estimate_4class_with_test.tar.gz ./

## 5 Class

In [ ]:
five_classes = Classifier("/content/pose_estimate", ["up", "down", "full", "right", "left"], "skeleton_5.h5")
five_classes.move_files()
five_classes.preprocesss()
five_classes.visualize()
five_classes.train()
five_classes.evaluate("/content/test/346.jpg", "full")
five_classes.test("/content/test/384.jpg") #up
five_classes.test("/content/test/5.jpg") #down
five_classes.test("/content/test/343.jpg") #full
five_classes.test("/content/test/501.jpg") #right
five_classes.test("/content/test/265.jpg") #left
five_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_5class_with_test.tar.gz ./skeleton_5.h5 ./test/
!rm -r /content/skeleton_5.h5

In [ ]:
# UNZIP FILES
!tar -xvzf pose_estimate_5class_with_test.tar.gz ./

## 6 Class

In [ ]:
six_classes = Classifier("/content/pose_estimate", ["up", "down", "full", "right", "left", "rightup"], "skeleton_6.h5")
six_classes.move_files()
six_classes.preprocesss()
six_classes.visualize()
six_classes.train()
six_classes.evaluate("/content/test/346.jpg", "full")
six_classes.test("/content/test/384.jpg") #up
six_classes.test("/content/test/5.jpg") #down
six_classes.test("/content/test/343.jpg") #full
six_classes.test("/content/test/501.jpg") #right
six_classes.test("/content/test/265.jpg") #left
six_classes.test("/content/test/405.jpg") #rightup
six_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_6class_with_test.tar.gz ./skeleton_6.h5 ./test/
!rm -r /content/skeleton_6.h5

# UNZIP FILES
!tar -xvzf pose_estimate_6class_with_test.tar.gz ./

## 7 Class

In [ ]:
seven_classes = Classifier("/content/pose_estimate", ["up", "down", "full", "right", "left", "rightup", "leftup"], "skeleton_7.h5")
seven_classes.move_files()
seven_classes.preprocesss()
seven_classes.visualize()
seven_classes.train()
seven_classes.evaluate("/content/test/346.jpg", "full")
seven_classes.test("/content/test/384.jpg") #up
seven_classes.test("/content/test/5.jpg") #down
seven_classes.test("/content/test/343.jpg") #full
seven_classes.test("/content/test/501.jpg") #right
seven_classes.test("/content/test/265.jpg") #left
seven_classes.test("/content/test/405.jpg") #rightup
seven_classes.test("/content/test/597.jpg") #leftup
seven_classes.remove_files()

In [ ]:
# ZIP FILES
!tar -czvf pose_estimate_7class_with_test.tar.gz ./skeleton_7.h5 ./test/
!rm -r /content/skeleton_7.h5

# UNZIP FILES
!tar -xvzf pose_estimate_7class_with_test.tar.gz ./

In [ ]:
265 555 561 630 641 left
381 384 100 up
394 405 rightup
463 501 right
577 582 597 leftup
5 518 662 down
346 343 full